In [ ]:
from tensorflow.keras import Sequential, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Bidirectional, Dense, Embedding, LSTM
from tensorflow.keras.saving import load_model, save_model

In [ ]:
class Bidirectional_LSTM:
    def __init__(self, X_train, y_train, embeddings_matrix, n_hidden_units=[10, 20, 10], dropuot_fraction=0.3, 
                 epochs=500, batch_size=200, validation_fraction=0.2):
        self.X_train = X_train
        self.y_train = y_train
        self.embeddings_matrix = embeddings_matrix

        self.n_hidden_units = n_hidden_units
        self.dropuot_fraction = dropuot_fraction
        self.epochs = epochs
        self.batch_size = batch_size
        self.validation_fraction = validation_fraction

        self.__init_model()

    def __init_model(self):
        # Bidirectional LSTM
        self.model = Sequential(name='Bidirectional_LSTM')
        
        # input layer
        self.model.add(Embedding(input_dim=self.embeddings_matrix.shape[0],
                            output_dim=self.embeddings_matrix.shape[1],
                            embeddings_initializer=Constant(self.embeddings_matrix),
                            mask_zero=True,
                            sparse=False))
    # hidden layer
        for n_units in self.n_hidden_units[:-1]:
            self.__add_bidirectional_layer(n_units, return_sequences=True)
        
        self.__add_bidirectional_layer(self.n_hidden_units[-1], return_sequences=False)

        # output layer
        self.model.add(Dense(units=1,
                        activation='sigmoid'))
        
        self.model.compile(loss='binary_crossentropy', 
                    optimizer='adam', 
                    metrics=['accuracy'])
    
    def __add_bidirectional_layer(self, n_units, return_sequences):
        # forward layer
        forward = LSTM(units=n_units,
                    activation="tanh",
                    recurrent_activation="sigmoid",
                    dropout=self.dropuot_fraction,
                    recurrent_dropout=self.dropuot_fraction,
                    return_sequences=return_sequences)
        
        # backward layer
        backward = LSTM(units=n_units,
                    activation="tanh",
                    recurrent_activation="sigmoid",
                    dropout=self.dropuot_fraction,
                    recurrent_dropout=self.dropuot_fraction,
                    return_sequences=return_sequences,
                    go_backwards=True)

        self.model.add(Bidirectional(forward, backward_layer=backward))

    def train(self):
        # early_stopping
        callbacks = EarlyStopping(monitor="val_loss",
                                min_delta=1e-12,
                                patience=2,
                                verbose=0,
                                mode="auto",
                                baseline=None,
                                restore_best_weights=False,
                                start_from_epoch=0)

        steps_per_epoch = int(self.X_train.shape[0] * 0.3 / self.batch_size)

        self.history = self.model.fit(x=np.array(self.X_train),
                                y=np.array(self.y_train),
                                batch_size=self.batch_size,
                                epochs=self.epochs,
                                verbose="auto",
                                callbacks=[callbacks],
                                validation_split=self.validation_fraction, # validation size
                                shuffle=True,
                                steps_per_epoch=steps_per_epoch).__dict__
        self.accuracy = self.model.evaluate(x=self.X_train,
                                y=self.y_train,
                                batch_size=self.batch_size,
                                verbose="auto",
                                return_dict=False) # metric return as a list
    
    def show_learning_curve(self):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

        # error
        ax1.plot(self.history['history']['loss'], label="Training error", color="red")
        if self.validation_fraction > 0:
            ax1.plot(self.history['history']['val_loss'], label="Validation error", color="blue")

        ax1.legend()
        ax1.grid()
        ax1.set(xlabel="Epochs", ylabel="Reconstruction Errors (binary crossentropy)")
        ax1.set_title("Reconstruction Errors")
        
        # accuracy
        ax2.plot(self.history['history']['accuracy'], label="Training accuracy", color="red")
        if self.validation_fraction > 0:
            ax2.plot(self.history['history']['val_accuracy'], label="Validation accuracy", color="blue")

        ax2.legend()
        ax2.grid()
        ax2.set(xlabel="Epochs", ylabel="Accuracy")
        ax2.set_title("Accuracy")
    
    def predict(self, X_test, batch_size=None):
        return np.where(self.model.predict(X_test, batch_size=batch_size) < 0.5, 0, 1)
    
    def compute_prediction_score(self, X_test, y_test, evaluation_metric='accuracy', batch_size=None):
        return self.model.evaluate(X_test, y_test, batch_size=batch_size, return_dict=True)[evaluation_metric]

    def save_model_history(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump([self.history['history'], self.accuracy], file)
            

In [ ]:
def load_model_history(filename):    
    with open(filename, 'rb') as file:
        [history, accuracy] = pickle.load(file)
            
    return history, accuracy

In [ ]:
model = Bidirectional_LSTM(X_train, y_train, embeddings_matrix=embeddings_matrix, n_hidden_units=[2, 2], 
                                   dropuot_fraction=0.3, epochs=30, batch_size=4)
        
model.train()

filename = ('models/bi_lstm.pkl' %i)
model.save_model_history(filename)

In [ ]:
show_learning_curve()